In [1]:
import config, data_load, data_info, data_plot
import plotly.express as px
import seaborn as sns
import sqlite3
import pandas as pd
import numpy as np

In [47]:
path_query = 'queries/check/partos.sql'
with open(path_query) as f:
	query = f.read()
df_partos = data_load.df_query(query)
df_partos

,def_procedimento_realizado,records
0,PARTO NORMAL,10410614
1,PARTO CESARIANO,6405986
2,PARTO CESARIANO EM GESTACAO DE ALTO RISCO,1219953
3,PARTO NORMAL EM GESTACAO DE ALTO RISCO,791773
4,PARTO CESARIANO C/ LAQUEADURA TUBARIA,284565
5,PARTO NORMAL EM CENTRO DE PARTO NORMAL (CPN),73667


In [45]:
olhando = df_partos['records'][:2].sum()
descarte = df_partos['records'][2:].sum()
100 - (olhando / (olhando + descarte))

99.12352117169333

In [46]:
print(olhando)
print(descarte)

16816760
2369967


In [18]:
path_query = 'queries/check/sinasc.sql'
with open(path_query) as f:
	query = f.read()
df_nasc = data_load.df_query(query)
df_nasc

,CODMUNNASC,records
0,355030,2836381
1,3550308,2082106
2,330455,1335522
3,530010,839282
4,310620,713943
...,...,...
995,2105708,12967
996,420750,12952
997,431800,12948
998,353390,12946


In [19]:
df['CODMUNRES'].apply(lambda x: len(str(x))).value_counts()

6    587
7    413
Name: CODMUNRES, dtype: int64

In [21]:
df_nasc['CODMUNNASC'].apply(lambda x: len(str(x))).value_counts()

6    577
7    423
Name: CODMUNNASC, dtype: int64

In [10]:
74398079-74398079

0

In [2]:
cnx = sqlite3.connect(config.PATH_DB)

In [35]:
path_filters = 'queries/info/check.sql'
with open(path_filters) as f:
	query_filters = f.read()
df = data_load.df_query(query_filters)
df

In [3]:
path_year_percent = 'queries/info/grouped.sql'
with open(path_year_percent) as f:
	query = f.read()
df = pd.read_sql_query(query, cnx)
df

,ano,parto,regiao,uf,grupo,mov_municipio,mov_regiao_saude,records
0,2010,NORMAL,Sudeste,SP,2,0,0,81325
1,2010,CESARIANO,Sudeste,SP,2,0,0,35302
2,2010,NORMAL,Sudeste,SP,1,0,0,34039
3,2010,NORMAL,Nordeste,MA,1,0,0,32523
4,2010,NORMAL,Sudeste,RJ,3,0,0,31756
...,...,...,...,...,...,...,...,...
7458,2019,NORMAL,Norte,AC,5,1,1,2
7459,2019,NORMAL,Norte,RO,4,1,1,2
7460,2019,NORMAL,Norte,RR,2,1,1,2
7461,2019,CESARIANO,Norte,RO,4,1,1,1


In [4]:
def group_all_but(df, but='records'):
	return df.groupby(
		by=[col for col in df.columns if col != but],
		as_index=False
	).sum().sort_values(
		by=['ano', 'records'],
		ascending=[True, False],
	)

In [5]:
def get_mov_between(df, mov_between):
	cols = [col
		for col in df.columns if col[:3] != 'mov']
	dff = df[cols].copy()
	dff['mov_between'] = mov_between
	dff = group_all_but(dff)
	return dff

In [6]:
def group_mov_between(df):
	return pd.concat([
		get_mov_between(df, 'all'),
		get_mov_between(df[df['mov_municipio'] == 1], 'municipal'),
		get_mov_between(df[df['mov_regiao_saude'] == 1], 'regional')
	]).reset_index(drop=True)

In [7]:
def separate_movs_in_dict(df):
	d = dict()
	movs = df['mov_between'].unique()
	cols = [col for col in df.columns if col[:3] != 'mov']
	for mov in movs:
		dff = df[
			df['mov_between'] == mov
		][cols].set_index(['ano', 'parto'])
		d[mov] = dff
	return d

In [8]:
def join_dfs(d, left, right):
	df = d[left].join(
		d[right],
		lsuffix='_num',
		rsuffix='_den',
	)
	df['mov_between'] = left
	df['percent'] = 100. * df['records_num'] / df['records_den']
	df = df.reset_index()
	return df

In [9]:
def get_frac(df):
	d = separate_movs_in_dict(df)
	return pd.concat([
		join_dfs(d, 'municipal', 'all'),
		join_dfs(d, 'regional', 'municipal')
	])

In [10]:
def group_col(df, column=None, value=None):
	del_cols = ['regiao', 'uf', 'grupo']
	if column:
		del_cols = [col
			for col in del_cols
				if col != column]
	cols_filter = [col for col in df.columns
			if col not in del_cols]
	dff = df[cols_filter].copy()
	if value:
		dff = dff[dff[column] == value]
	dff = group_all_but(dff)
	return get_frac(dff)

In [11]:
df_s = group_mov_between(df)
df_s

,ano,parto,regiao,uf,grupo,mov_between,records
0,2010,NORMAL,Sudeste,SP,2,all,86899
1,2010,NORMAL,Sudeste,SP,1,all,42266
2,2010,NORMAL,Nordeste,BA,1,all,40926
3,2010,CESARIANO,Sudeste,SP,2,all,38797
4,2010,NORMAL,Nordeste,MA,1,all,36604
...,...,...,...,...,...,...,...
7610,2019,NORMAL,Norte,AC,5,regional,2
7611,2019,NORMAL,Norte,RO,4,regional,2
7612,2019,NORMAL,Norte,RR,2,regional,2
7613,2019,CESARIANO,Norte,RO,4,regional,1


In [12]:
def plot(df):
	return px.scatter(df,
		x='ano',
		y='percent',
		color='parto',
		symbol='mov_between',
		trendline="ols",
	)

In [13]:
df = group_col(df_s,
	column = 'grupo',
	value  = 5,
)
fig = plot(df)
fig.show()

In [14]:
results = px.get_trendline_results(fig)
results['px_fit_results'][0].params

array([-1.00026217e+03,  5.09885052e-01])

In [15]:
dt = pd.DataFrame([r.params for r in results['px_fit_results']])
dt

,0,1
0,-1000.262167,0.509885
1,-556.269282,0.287995
2,-569.622453,0.297477
3,-197.884087,0.109798


In [16]:
px.scatter(dt,x=1,y=0)

In [17]:
for r in results:
	print(r)

parto
mov_between
px_fit_results


In [18]:
df
fig = plot(df)
fig.show()

In [19]:
df = group_col(df_s,
	column = 'regiao',
	value  = 'Norte',
)

In [20]:
df
fig = plot(df)
fig.show()

In [21]:
df = group_col(df_s,
	column = 'regiao',
	value  = None,
)
df

,ano,parto,regiao_num,records_num,regiao_den,records_den,mov_between,percent
0,2010,CESARIANO,Nordeste,80764,Sudeste,209153,municipal,38.614794
1,2010,CESARIANO,Nordeste,80764,Nordeste,202687,municipal,39.846660
2,2010,CESARIANO,Nordeste,80764,Sul,79104,municipal,102.098503
3,2010,CESARIANO,Nordeste,80764,Norte,69379,municipal,116.409865
4,2010,CESARIANO,Nordeste,80764,Centro-Oeste,52453,municipal,153.974034
...,...,...,...,...,...,...,...,...
495,2019,NORMAL,Sul,3859,Nordeste,106610,regional,3.619735
496,2019,NORMAL,Sul,3859,Sudeste,71948,regional,5.363596
497,2019,NORMAL,Sul,3859,Centro-Oeste,50447,regional,7.649612
498,2019,NORMAL,Sul,3859,Sul,34509,regional,11.182590
